# Import the necessary Libraries

In [259]:
import pandas as pd
import numpy as np
import random
import requests
import time

## To start our analysis and to comply with compute restrictions I will first analyse the data for a total of 5 companies out of 500

I will first start by studying 5 companies in the same industry so to have a more industry specific insights. Given the companies in that industry I will know the patterns that emerge in that specific industry and from that point on, when I have completed that industry I can generalize it to other companies in the same industry at first and than into other industries

### Advantages:
1. **Domain-Specific patterns**
2. Comparable analysis
3. Controlled complexity
4. Faster iteration
5. Clear baseline

### Information Techonology Industry
1. **Microsoft** 
2. **ServiceNow**
3. **AMD**
4. **Salesforce**
5. **Palantir**

### I am downloading the MSFT financial data
- Import the Financial csv data
- sort values by date
- Pivot the table to make the concept appear as independant columns across different date&Time
- Given that during the pivote there were a lot of duplicate that were generated, I group the columns by filed date and remove any NaN cells as to have a cleaned dataframe
- To further clean the data, I removed the columns that are 95% empty. I cannot do any meaning full Time Series analysis on those columns

In [260]:
df = pd.read_csv('./data/financial_data/MSFT_raw_financials.csv')

In [261]:

pivoted_df = df.pivot_table(
    index=['cik', 'company_name', 'frame','unit', 'end', 'filed', 'form'],
    columns='concept',
    values='value'
).reset_index()

pivoted_df.head(2)

concept,cik,company_name,frame,unit,end,filed,form,AccountsPayableCurrent,AccountsReceivableNet,AccountsReceivableNetCurrent,...,UnrecognizedTaxBenefitsDecreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestExpense,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,789019,MICROSOFT CORPORATION,CY2007Q2I,USD,2007-06-30,2010-07-30,10-K,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,789019,MICROSOFT CORPORATION,CY2007Q2I,shares,2007-06-30,2010-07-30,10-K,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Code that groups by filed column and remove most of the NaN cells. This way I will have all the relavant columns and remove any duplicates and NaN values**

In [262]:
# Define metadata columns to preserve
metadata_cols = ['cik', 'company_name', 'form', 'frame', 'end', 'unit']

# Create an empty DataFrame to store results
consolidated_df = pd.DataFrame()

# Group by filed date
grouped = pivoted_df.groupby('end')

# Iterate through each group
for filed_date, group in grouped:
    # Create a dictionary to store the row data
    row_data = {'end': filed_date}
    
    # For metadata columns, just take the first value
    for col in metadata_cols:
        if col in pivoted_df.columns:
            row_data[col] = group[col].iloc[0]
    
    # For all other columns, take the first non-NaN value
    for col in pivoted_df.columns:
        if col not in metadata_cols and col != 'end':
            non_nan_values = group[col].dropna()
            if not non_nan_values.empty:
                row_data[col] = non_nan_values.iloc[0]
            else:
                row_data[col] = None
    
    # Append this row to our result DataFrame
    consolidated_df = pd.concat([consolidated_df, pd.DataFrame([row_data])], ignore_index=True)

# Display the first few rows to verify
consolidated_df.head()

/var/folders/0w/q70p80s12nz5lpzr087614lw0000gn/T/ipykernel_4718/41078436.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  consolidated_df = pd.concat([consolidated_df, pd.DataFrame([row_data])], ignore_index=True)


,end,cik,company_name,form,frame,unit,filed,AccountsPayableCurrent,AccountsReceivableNet,AccountsReceivableNetCurrent,...,UnrecognizedTaxBenefitsDecreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestExpense,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,2007-06-30,789019,MICROSOFT CORPORATION,10-K,CY2007Q2I,USD,2010-07-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-09-30,789019,MICROSOFT CORPORATION,10-K,CY2007Q3,USD,2010-07-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-12-31,789019,MICROSOFT CORPORATION,10-K,CY2007Q4,USD,2010-07-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-03-31,789019,MICROSOFT CORPORATION,10-K,CY2008Q1,USD,2010-07-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-06-30,789019,MICROSOFT CORPORATION,10-K,CY2008,USD,2010-07-30,NaN,NaN,NaN,...,80000000.0,4.787000e+09,324000000.0,121000000.0,934000000.0,66000000.0,14000000.0,NaN,9.470000e+09,9.328000e+09


In [263]:
# Filter the DataFrame to keep only rows where year in 'filed' column is 2020 or later
consolidated_df = consolidated_df[consolidated_df['filed'].str[:4].astype(int) >= 2019]
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
consolidated_df = consolidated_df[~consolidated_df['frame'].str.match(r'CY\d{4}Q[1-4].+')]

**In the following function I am computing the number of empty cells in each columns and removing all the columns that are 95% empty**

In [264]:
# Calculate the percentage of NaN values in each column
nan_percentages = consolidated_df.isna().mean() * 100

# Identify columns with 95% or more NaN values
columns_to_drop = nan_percentages[nan_percentages >= 85].index.tolist()

# Display columns that will be dropped
print(f"Columns being dropped (>= 95% empty): {columns_to_drop}")
print(f"Number of columns before: {consolidated_df.shape[1]}")

# Drop the identified columns
consolidated_df = consolidated_df.drop(columns=columns_to_drop)

# Display the new shape
print(f"Number of columns after: {consolidated_df.shape[1]}")

Columns being dropped (>= 95% empty): ['AccountsReceivableNet', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AcquiredFiniteLivedIntangibleAssetAmount', 'AcquiredFiniteLivedIntangibleAssetWeightedAverageUsefulLife', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AssetImpairmentCharges', 'AssetsFairValueDisclosureRecurring', 'AvailableForSaleSecurities', 'AvailableForSaleSecuritiesAccumulatedGrossUnrealizedGainBeforeTax', 'AvailableForSaleSecuritiesAccumulatedGrossUnrealizedLossBeforeTax', 'AvailableForSaleSecuritiesAmortizedCost', 'AvailableForSaleSecuritiesContinuousUnrealizedLossPosition12MonthsOrLongerAccumulatedLoss', 'AvailableForSaleSecuritiesContinuousUnrealizedLossPositionAccumulatedLoss', 'AvailableForSaleSecuritiesConti

In [265]:
# Display the first few rows of the filtered dataframe
k.tail(30)

,end,cik,company_name,form,frame,unit,filed,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccountsReceivableNetNoncurrent,...,UnrecognizedTaxBenefitsDecreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestExpense,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
50,2017-05-01,789019,MICROSOFT CORPORATION,10-K,CY2017Q1I,USD,2019-08-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2017-09-30,789019,MICROSOFT CORPORATION,10-K,CY2017Q3,USD,2019-08-01,6.866000e+09,1.456100e+10,1.600000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.799000e+09,7.708000e+09
53,2017-12-31,789019,MICROSOFT CORPORATION,10-K,CY2017Q4,USD,2019-08-01,7.850000e+09,1.842800e+10,1.600000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.710000e+09,7.710000e+09
54,2018-03-31,789019,MICROSOFT CORPORATION,10-K,CY2018Q1,USD,2019-08-01,7.623000e+09,1.720800e+10,1.600000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794000e+09,7.698000e+09
55,2018-05-01,789019,MICROSOFT CORPORATION,10-K,CY2018Q1I,USD,2020-07-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,2018-09-30,789019,MICROSOFT CORPORATION,10-K,CY2018Q3,USD,2020-07-31,8.511000e+09,1.739000e+10,1.800000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.766000e+09,7.673000e+09
58,2018-12-31,789019,MICROSOFT CORPORATION,10-K,CY2018Q4,USD,2020-07-31,7.563000e+09,1.968000e+10,1.900000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.768000e+09,7.692000e+09
59,2019-03-31,789019,MICROSOFT CORPORATION,10-K,CY2019Q1,USD,2020-07-31,7.544000e+09,1.926900e+10,2.000000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.744000e+09,7.672000e+09
60,2019-05-01,789019,MICROSOFT CORPORATION,10-K,CY2019Q1I,USD,2021-07-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,2019-06-30,789019,MICROSOFT CORPORATION,10-K,CY2019,USD,2019-08-01,9.382000e+09,2.952400e+10,2.200000e+09,...,1.113000e+09,316000000.0,3.400000e+09,515000000.0,2.106000e+09,508000000.0,0.0,1.200000e+10,7.753000e+09,7.673000e+09


# Balance Sheet for 10-K

In [266]:
import pandas as pd
import numpy as np

def reconstruct_balance_sheet(df):
    """
    Reconstructs balance sheet from XBRL-tagged data.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed balance sheet for each period
    """
    # Create a new DataFrame to store the reconstructed balance sheet
    balance_sheet = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            balance_sheet[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for balance sheet accounts based on XBRL structure
    
    # Assets section
    assets = {
        # Current Assets
        'Cash and Cash Equivalents': {
            'primary': 'CashAndCashEquivalents',
            'alternatives': ['CashAndCashEquivalentsAtCarryingValue'],
            'children': []
        },
        'Short-term Investments': {
            'primary': 'ShortTermInvestments',
            'alternatives': ['MarketableSecuritiesCurrent'],
            'children': ['AvailableForSaleSecuritiesCurrent', 'InvestmentsAvailableForSaleCurrent']
        },
        'Accounts Receivable, Net (Current)': {
            'primary': 'AccountsReceivableNetCurrent',
            'alternatives': [],
            'children': []
        },
        'Accounts Receivable, Net (Non-current)': {
            'primary': 'AccountsReceivableNetNoncurrent',
            'alternatives': [],
            'children': []
        },
        'Inventory': {
            'primary': 'InventoryNet',
            'alternatives': ['Inventory'],
            'children': []
        },
        'Other Current Assets': {
            'primary': 'OtherAssetsCurrent',
            'alternatives': ['OtherCurrentAssets'],
            'children': ['PrepaidExpensesCurrent']
        },
        'Total Current Assets': {
            'primary': 'AssetsCurrent',
            'alternatives': ['CurrentAssets'],
            'children': []
        },
        
        # Non-current Assets
        'Property, Plant and Equipment, Net': {
            'primary': 'PropertyPlantAndEquipmentNet',
            'alternatives': ['PropertyAndEquipmentNet'],
            'children': []
        },
        'Operating Lease Right-of-Use Assets': {
            'primary': 'OperatingLeaseRightOfUseAssets',
            'alternatives': [],
            'children': []
        },
        'Equity Investments': {
            'primary': 'EquityInvestments',
            'alternatives': [],
            'children': []
        },
        'Goodwill': {
            'primary': 'Goodwill',
            'alternatives': [],
            'children': []
        },
        'Intangible Assets, Net': {
            'primary': 'IntangibleAssetsNetExcludingGoodwill',
            'alternatives': ['IntangibleAssetsNet'],
            'children': []
        },
        'Other Long-term Assets': {
            'primary': 'OtherAssetsNoncurrent',
            'alternatives': ['OtherNoncurrentAssets', 'OtherLongTermAssets'],
            'children': ['DeferredIncomeTaxesNoncurrent']
        },
        'Total Assets': {
            'primary': 'Assets',
            'alternatives': [],
            'children': []
        }
    }
    
    # Liabilities section
    liabilities = {
        # Current Liabilities
        'Accounts Payable': {
            'primary': 'AccountsPayableCurrent',
            'alternatives': ['AccountsPayable'],
            'children': []
        },
        'Current Portion of Long-term Debt': {
            'primary': 'LongTermDebtCurrent',
            'alternatives': ['DebtCurrent'],
            'children': []
        },
        'Accrued Compensation': {
            'primary': 'AccruedCompensation',
            'alternatives': [],
            'children': []
        },
        'Income Taxes Payable': {
            'primary': 'IncomeTaxesPayableCurrent',
            'alternatives': ['IncomeTaxPayableCurrent'],
            'children': []
        },
        'Unearned Revenue (Current)': {
            'primary': 'DeferredRevenueCurrent',
            'alternatives': ['UnearnedRevenueCurrent', 'ContractLiabilitiesCurrent'],
            'children': []
        },
        'Other Current Liabilities': {
            'primary': 'OtherLiabilitiesCurrent',
            'alternatives': ['OtherCurrentLiabilities', 'OtherAccruedLiabilitiesCurrent'],
            'children': []
        },
        'Total Current Liabilities': {
            'primary': 'LiabilitiesCurrent',
            'alternatives': ['CurrentLiabilities'],
            'children': []
        },
        
        # Non-current Liabilities
        'Long-term Debt': {
            'primary': 'LongTermDebtNoncurrent',
            'alternatives': ['DebtNoncurrent', 'LongTermDebt'],
            'children': []
        },
        'Long-term Income Taxes': {
            'primary': 'IncomeTaxesPayableNoncurrent',
            'alternatives': ['IncomeTaxPayableNoncurrent', 'LongTermIncomeTaxPayable'],
            'children': []
        },
        'Long-term Unearned Revenue': {
            'primary': 'DeferredRevenueNoncurrent',
            'alternatives': ['UnearnedRevenueNoncurrent', 'ContractLiabilitiesNoncurrent'],
            'children': []
        },
        'Operating Lease Liabilities (Non-current)': {
            'primary': 'OperatingLeaseLiabilityNoncurrent',
            'alternatives': [],
            'children': []
        },
        'Other Long-term Liabilities': {
            'primary': 'OtherLiabilitiesNoncurrent',
            'alternatives': ['OtherNoncurrentLiabilities', 'OtherLongTermLiabilities'],
            'children': []
        },
        'Total Liabilities': {
            'primary': 'Liabilities',
            'alternatives': [],
            'children': []
        }
    }
    
    # Equity section
    equity = {
        'Common Stock and Paid-in Capital': {
            'primary': 'CommonStockAndAdditionalPaidInCapital',
            'alternatives': [],
            'children': ['CommonStockValue', 'AdditionalPaidInCapital']
        },
        'Retained Earnings': {
            'primary': 'RetainedEarningsAccumulatedDeficit',
            'alternatives': ['RetainedEarnings'],
            'children': []
        },
        'Accumulated Other Comprehensive Income': {
            'primary': 'AccumulatedOtherComprehensiveIncomeLoss',
            'alternatives': [],
            'children': []
        },
        'Total Stockholders Equity': {
            'primary': 'StockholdersEquity',
            'alternatives': ['TotalEquity'],
            'children': []
        },
        'Total Liabilities and Equity': {
            'primary': 'LiabilitiesAndStockholdersEquity',
            'alternatives': ['LiabilitiesAndEquity'],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        for col in available_cols:
            if account_info['primary'] in col:
                return col
                
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Try to find the best match
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                balance_sheet[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    balance_sheet[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    balance_sheet[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                balance_sheet[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Assets", assets)
    process_section("Liabilities", liabilities)
    process_section("Equity", equity)
    
    # Add validation checks - this will be updated in the add_total_liabilities_and_equity function
    
    # Check if current assets and liabilities match their components
    if 'Assets - Total Current Assets' in balance_sheet.columns:
        current_asset_cols = [col for col in balance_sheet.columns if col.startswith('Assets - ') and 'Current' in col and 'Total' not in col]
        if current_asset_cols:
            balance_sheet['Validation - Current Assets Difference'] = balance_sheet['Assets - Total Current Assets'] - balance_sheet[current_asset_cols].sum(axis=1)
    
    if 'Liabilities - Total Current Liabilities' in balance_sheet.columns:
        current_liab_cols = [col for col in balance_sheet.columns if col.startswith('Liabilities - ') and 'Current' in col and 'Total' not in col]
        if current_liab_cols:
            balance_sheet['Validation - Current Liabilities Difference'] = balance_sheet['Liabilities - Total Current Liabilities'] - balance_sheet[current_liab_cols].sum(axis=1)
    
    return balance_sheet

# Function to display a human-readable balance sheet
def display_balance_sheet(balance_sheet, in_billions=True):
    """
    Display the balance sheet in a clean, readable format.
    
    Args:
        balance_sheet: DataFrame with the reconstructed balance sheet
        in_billions: If True, display values in billions; otherwise in millions
    
    Returns:
        DataFrame with the formatted balance sheet
    """
    # Make a copy to avoid modifying the original
    formatted_bs = balance_sheet.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_bs.columns if any(col.startswith(prefix) for prefix in ['Assets', 'Liabilities', 'Equity', 'Validation'])]
    
    # Convert to billions or millions
    divisor = 1_000_000_000 if in_billions else 1_000_000
    suffix = 'B' if in_billions else 'M'
    
    for col in numeric_cols:
        formatted_bs[col] = formatted_bs[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_bs.columns:
        try:
            formatted_bs['end'] = pd.to_datetime(formatted_bs['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display
    asset_cols = [col for col in formatted_bs.columns if col.startswith('Assets')]
    liability_cols = [col for col in formatted_bs.columns if col.startswith('Liabilities')]
    equity_cols = [col for col in formatted_bs.columns if col.startswith('Equity')]
    validation_cols = [col for col in formatted_bs.columns if col.startswith('Validation')]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_bs.columns else []
    display_cols.extend(asset_cols + liability_cols + equity_cols + validation_cols)
    
    return formatted_bs[display_cols]

# Function to add Total Liabilities and Equity if it doesn't exist
def add_total_liabilities_and_equity(balance_sheet):
    """
    Adds Total Liabilities and Equity column if it doesn't exist already.
    Ensures it equals Total Assets for validation.
    
    Args:
        balance_sheet: DataFrame with the reconstructed balance sheet
    
    Returns:
        DataFrame with Total Liabilities and Equity added
    """
    if 'Equity - Total Liabilities and Equity' not in balance_sheet.columns:
        if 'Liabilities - Total Liabilities' in balance_sheet.columns and 'Equity - Total Stockholders\' Equity' in balance_sheet.columns:
            balance_sheet['Equity - Total Liabilities and Equity'] = (
                balance_sheet['Liabilities - Total Liabilities'] + 
                balance_sheet['Equity - Total Stockholders\' Equity']
            )
    
    # Add validation check to ensure Total Assets = Total Liabilities and Equity
    if 'Assets - Total Assets' in balance_sheet.columns and 'Equity - Total Liabilities and Equity' in balance_sheet.columns:
        balance_sheet['Validation - A = L+E Difference'] = (
            balance_sheet['Assets - Total Assets'] - 
            balance_sheet['Equity - Total Liabilities and Equity']
        )
    
    return balance_sheet

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)

# Example usage
def analyze_microsoft_balance_sheet(df):
    """
    Analyze Microsoft's balance sheet from XBRL data.
    
    Args:
        df: DataFrame containing Microsoft's XBRL-tagged financial data
    
    Returns:
        The reconstructed balance sheet
    """
    print("Reconstructing Microsoft's balance sheet from XBRL data...")
    balance_sheet = reconstruct_balance_sheet(df)
    
    # Add Total Liabilities and Equity
    balance_sheet = add_total_liabilities_and_equity(balance_sheet)
    
    # Remove columns with only NaN values
    balance_sheet = remove_nan_only_columns(balance_sheet)
    
    print("\nBalance Sheet Summary:")
    print(f"- Total columns: {len(balance_sheet.columns)}")
    print(f"- Data for {len(balance_sheet)} reporting periods")
    
    # Display the balance sheet
    print("\nFormatted Balance Sheet (in billions USD):")
    formatted_bs = display_balance_sheet(balance_sheet, in_billions=True)
    
    return balance_sheet, formatted_bs

# To use this code with your DataFrame:
balance_sheet, formatted_bs = analyze_microsoft_balance_sheet(consolidated_df)

Reconstructing Microsoft's balance sheet from XBRL data...

Balance Sheet Summary:
- Total columns: 35
- Data for 29 reporting periods

Formatted Balance Sheet (in billions USD):


In [267]:
yearly = balance_sheet[balance_sheet['frame'].str.match(r'^CY\d{4}$')]
yearly.T

,61,66,71,76,82,87
filed,2019-08-01,2020-07-31,2021-07-29,2023-07-27,2023-07-27,2024-12-03
company_name,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
end,2019-06-30,2020-06-30,2021-06-30,2022-06-30,2023-06-30,2024-06-30
unit,USD,USD,USD,USD,USD,USD
form,10-K,10-K,10-K,10-K,10-K,8-K
frame,CY2019,CY2020,CY2021,CY2022,CY2023,CY2024
cik,789019,789019,789019,789019,789019,789019
Assets - Cash and Cash Equivalents,11356000000.0,13576000000.0,14224000000.0,13931000000.0,34704000000.0,18315000000.0
Assets - Short-term Investments,122463000000.0,122951000000.0,116110000000.0,90826000000.0,76558000000.0,57228000000.0
"Assets - Accounts Receivable, Net (Current)",29524000000.0,32011000000.0,38043000000.0,44261000000.0,48688000000.0,56924000000.0


# Balance Sheet after removing the columns that were not usefull for our workflow

In [268]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
balance_sheet = balance_sheet[~balance_sheet['frame'].str.match(r'CY\d{4}Q[1-4].+')]
balance_sheet = balance_sheet.sort_values(by='end')
balance_sheet.T

,52,53,54,57,58,59,61,62,63,64,...,77,78,80,82,83,84,85,87,88,89
filed,2019-08-01,2019-08-01,2019-08-01,2020-07-31,2020-07-31,2020-07-31,2019-08-01,2021-07-29,2021-07-29,2021-07-29,...,2024-12-03,2023-07-27,2024-12-03,2023-07-27,2023-10-24,2024-01-30,2024-12-03,2024-12-03,2025-01-29,2025-01-29
company_name,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,...,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
end,2017-09-30,2017-12-31,2018-03-31,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,...,8-K,10-K,8-K,10-K,10-Q,10-Q,8-K,8-K,10-Q,10-Q
frame,CY2017Q3,CY2017Q4,CY2018Q1,CY2018Q3,CY2018Q4,CY2019Q1,CY2019,CY2019Q3,CY2019Q4,CY2020Q1,...,CY2022Q3,CY2022Q4,CY2023Q1,CY2023,CY2023Q3,CY2023Q4,CY2024Q1,CY2024,CY2024Q3,CY2024Q4
cik,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019,...,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019
Assets - Cash and Cash Equivalents,6884000000.0,12859000000.0,9221000000.0,15137000000.0,6638000000.0,11212000000.0,11356000000.0,13117000000.0,8864000000.0,11710000000.0,...,22884000000.0,15646000000.0,26562000000.0,34704000000.0,80452000000.0,17305000000.0,19634000000.0,18315000000.0,20840000000.0,17482000000.0
Assets - Short-term Investments,NaN,NaN,NaN,120743000000.0,121024000000.0,120406000000.0,122463000000.0,123519000000.0,125389000000.0,125916000000.0,...,84378000000.0,83862000000.0,77865000000.0,76558000000.0,63499000000.0,63712000000.0,60387000000.0,57228000000.0,57588000000.0,54073000000.0
"Assets - Accounts Receivable, Net (Current)",14561000000.0,18428000000.0,17208000000.0,17390000000.0,19680000000.0,19269000000.0,29524000000.0,19087000000.0,23525000000.0,22699000000.0,...,31279000000.0,35833000000.0,37420000000.0,48688000000.0,36953000000.0,42831000000.0,44029000000.0,56924000000.0,44148000000.0,48188000000.0


In [269]:
balance_sheet.columns

Index(['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik',
       'Assets - Cash and Cash Equivalents', 'Assets - Short-term Investments',
       'Assets - Accounts Receivable, Net (Current)',
       'Assets - Accounts Receivable, Net (Non-current)', 'Assets - Inventory',
       'Assets - Other Current Assets', 'Assets - Total Current Assets',
       'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill',
       'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets',
       'Assets - Total Assets', 'Liabilities - Accounts Payable',
       'Liabilities - Current Portion of Long-term Debt',
       'Liabilities - Other Current Liabilities',
       'Liabilities - Total Current Liabilities',
       'Liabilities - Long-term Debt',
       'Liabilities - Operating Lease Liabilities (Non-current)',
       'Liabilities - Other Long-term Liabilities',
       'Liabilities - Total Liabilities',
       'Equity - Common Stock and Paid-in Capital',
       'Equity 

### Income statement

In [270]:
import pandas as pd
import numpy as np

def reconstruct_income_statement(df):
    """
    Reconstructs income statement from XBRL-tagged data, specifically tailored for IT companies like Microsoft.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed income statement for each period
    """
    # Create a new DataFrame to store the reconstructed income statement
    income_statement = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            income_statement[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for income statement accounts based on XBRL structure
    
    # Revenue section
    revenue = {
        'Total Revenue': {
            'primary': 'RevenueFromContractWithCustomerExcludingAssessedTax',
            'alternatives': ['SalesRevenueNet', 'Revenue', 'Revenues'],
            'children': ['ProductRevenue', 'ServiceRevenue', 'CloudComputingArrangementRevenue', 'SubscriptionServiceRevenue']
        },
        'Product Revenue': {
            'primary': 'ProductRevenue',
            'alternatives': ['SalesOfProductsNet'],
            'children': []
        },
        'Service Revenue': {
            'primary': 'ServiceRevenue',
            'alternatives': ['SalesOfServicesNet'],
            'children': []
        },
        'Cloud Revenue': {
            'primary': 'CloudComputingArrangementRevenue',
            'alternatives': ['HostingServiceRevenue', 'SubscriptionServiceRevenue'],
            'children': []
        }
    }
    
    # Cost of Revenue section
    cost_of_revenue = {
        'Total Cost of Revenue': {
            'primary': 'CostOfRevenue',
            'alternatives': ['CostOfGoodsSold', 'CostOfSales', 'CostOfGoodsAndServicesSold'],
            'children': ['CostOfProductRevenue', 'CostOfServiceRevenue']
        },
        'Cost of Product Revenue': {
            'primary': 'CostOfProductRevenue',
            'alternatives': ['CostOfGoodsSoldProduct'],
            'children': []
        },
        'Cost of Service Revenue': {
            'primary': 'CostOfServiceRevenue',
            'alternatives': ['CostOfServicesSold'],
            'children': []
        }
    }
    
    # Gross Profit
    gross_profit = {
        'Gross Profit': {
            'primary': 'GrossProfit',
            'alternatives': [],
            'children': []
        }
    }
    
    # Operating Expenses section
    operating_expenses = {
        'Research and Development': {
            'primary': 'ResearchAndDevelopmentExpense',
            'alternatives': ['ResearchDevelopmentExpense'],
            'children': []
        },
        'Sales and Marketing': {
            'primary': 'SalesAndMarketingExpense',
            'alternatives': ['MarketingExpense', 'SellingExpense', 'SellingAndMarketingExpense'],
            'children': []
        },
        'General and Administrative': {
            'primary': 'GeneralAndAdministrativeExpense',
            'alternatives': ['AdminExpense'],
            'children': []
        },
        'Selling, General and Administrative': {
            'primary': 'SellingGeneralAndAdministrativeExpenses',
            'alternatives': ['GeneralAndAdministrativeExpense'],
            'children': ['SalesAndMarketingExpense', 'GeneralAndAdministrativeExpense']
        },
        'Amortization of Intangible Assets': {
            'primary': 'AmortizationOfIntangibleAssets',
            'alternatives': ['AmortizationOfIntangibleAssetsExcludingGoodwill'],
            'children': []
        },
        'Restructuring Charges': {
            'primary': 'RestructuringCharges',
            'alternatives': ['RestructuringCosts'],
            'children': []
        },
        'Impairment of Goodwill': {
            'primary': 'ImpairmentOfGoodwill',
            'alternatives': [],
            'children': []
        },
        'Impairment of Intangible Assets': {
            'primary': 'ImpairmentOfIntangibleAssets',
            'alternatives': [],
            'children': []
        },
        'Total Operating Expenses': {
            'primary': 'OperatingExpenses',
            'alternatives': ['TotalOperatingExpenses'],
            'children': []
        }
    }
    
    # Operating Income
    operating_income = {
        'Operating Income': {
            'primary': 'IncomeLossFromOperations',
            'alternatives': ['OperatingIncomeLoss', 'IncomeFromOperations'],
            'children': []
        }
    }
    
    # Other Income/Expense section
    other_income_expense = {
        'Interest Income': {
            'primary': 'InterestIncome',
            'alternatives': ['InvestmentIncomeInterest'],
            'children': []
        },
        'Interest Expense': {
            'primary': 'InterestExpense',
            'alternatives': ['InterestCost'],
            'children': []
        },
        'Gain/Loss on Investments': {
            'primary': 'GainLossOnInvestments',
            'alternatives': ['InvestmentIncomeNetGainLossFromSaleOfInvestment'],
            'children': []
        },
        'Foreign Currency Transaction Gain/Loss': {
            'primary': 'ForeignCurrencyTransactionGainLoss',
            'alternatives': [],
            'children': []
        },
        'Other Income/Expense, Net': {
            'primary': 'OtherIncomeExpenseNet',
            'alternatives': ['NonoperatingIncomeExpense'],
            'children': []
        }
    }
    
    # Income before taxes
    income_before_tax = {
        'Income Before Income Taxes': {
            'primary': 'IncomeLossBeforeIncomeTax',
            'alternatives': ['IncomeBeforeTax', 'EarningsBeforeIncomeTax'],
            'children': []
        }
    }
    
    # Income tax provision
    income_tax = {
        'Income Tax Expense/Benefit': {
            'primary': 'IncomeTaxExpenseBenefit',
            'alternatives': ['IncomeTaxesPaidCurrent', 'IncomeTaxesPaid'],
            'children': []
        }
    }
    
    # Net Income
    net_income = {
        'Net Income': {
            'primary': 'ProfitLoss',
            'alternatives': ['NetIncomeLoss'],
            'children': []
        },
        'Net Income Attributable to Noncontrolling Interest': {
            'primary': 'NetIncomeLossAttributableToNoncontrollingInterest',
            'alternatives': [],
            'children': []
        },
        'Net Income Attributable to Company': {
            'primary': 'NetIncomeLossAttributableToParent',
            'alternatives': ['NetIncomeLossAvailableToCommonStockholdersBasic'],
            'children': []
        }
    }
    
    # Earnings Per Share
    eps = {
        'Basic EPS': {
            'primary': 'EarningsPerShareBasic',
            'alternatives': [],
            'children': []
        },
        'Diluted EPS': {
            'primary': 'EarningsPerShareDiluted',
            'alternatives': [],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        primary_matches = []
        for col in available_cols:
            if account_info['primary'] in col:
                primary_matches.append(col)
        
        # If we found exactly one partial match for primary, return it
        if len(primary_matches) == 1:
            return primary_matches[0]
                
        # If we found multiple partial matches, try to find the best one
        if primary_matches:
            # For certain important accounts, use a more specific matching strategy
            if account_info['primary'] in ['ProfitLoss', 'RevenueFromContractWithCustomer', 'CostOfRevenue']:
                for match in primary_matches:
                    if account_info['primary'] in match and not any(x in match.lower() for x in ['segment', 'division', 'product']):
                        return match
            
            return primary_matches[0]  # Default to first one
        
        # Check for partial matches in alternatives
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Try to find the best match
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                income_statement[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    income_statement[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    income_statement[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                income_statement[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Revenue", revenue)
    process_section("Cost", cost_of_revenue)
    process_section("Gross", gross_profit)
    process_section("Operating Expenses", operating_expenses)
    process_section("Operating", operating_income)
    process_section("Other", other_income_expense)
    process_section("Income Before Tax", income_before_tax)
    process_section("Tax", income_tax)
    process_section("Net Income", net_income)
    process_section("EPS", eps)
    
    # Calculate missing values and add validation checks
    
    # If Gross Profit is missing, calculate it
    if ('Gross - Gross Profit' not in income_statement.columns or income_statement['Gross - Gross Profit'].isna().all()) and \
       'Revenue - Total Revenue' in income_statement.columns and 'Cost - Total Cost of Revenue' in income_statement.columns:
        income_statement['Gross - Gross Profit (calculated)'] = income_statement['Revenue - Total Revenue'] - income_statement['Cost - Total Cost of Revenue']
    
    # If Operating Income is missing, calculate it
    if ('Operating - Operating Income' not in income_statement.columns or income_statement['Operating - Operating Income'].isna().all()) and \
       'Gross - Gross Profit' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns:
        income_statement['Operating - Operating Income (calculated)'] = income_statement['Gross - Gross Profit'] - income_statement['Operating Expenses - Total Operating Expenses']
    elif ('Operating - Operating Income' not in income_statement.columns or income_statement['Operating - Operating Income'].isna().all()) and \
         'Gross - Gross Profit (calculated)' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns:
        income_statement['Operating - Operating Income (calculated)'] = income_statement['Gross - Gross Profit (calculated)'] - income_statement['Operating Expenses - Total Operating Expenses']
    
    # If Income Before Tax is missing, calculate it
    if ('Income Before Tax - Income Before Income Taxes' not in income_statement.columns or income_statement['Income Before Tax - Income Before Income Taxes'].isna().all()):
        # Try to calculate from Operating Income and Other Income/Expense
        if 'Operating - Operating Income' in income_statement.columns and 'Other - Other Income/Expense, Net' in income_statement.columns:
            income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] = income_statement['Operating - Operating Income'] + income_statement['Other - Other Income/Expense, Net']
        elif 'Operating - Operating Income (calculated)' in income_statement.columns and 'Other - Other Income/Expense, Net' in income_statement.columns:
            income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] = income_statement['Operating - Operating Income (calculated)'] + income_statement['Other - Other Income/Expense, Net']
    
    # If Net Income is missing, calculate it
    if ('Net Income - Net Income' not in income_statement.columns or income_statement['Net Income - Net Income'].isna().all()):
        # Try to calculate from Income Before Tax and Income Tax Expense
        if 'Income Before Tax - Income Before Income Taxes' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns:
            income_statement['Net Income - Net Income (calculated)'] = income_statement['Income Before Tax - Income Before Income Taxes'] - income_statement['Tax - Income Tax Expense/Benefit']
        elif 'Income Before Tax - Income Before Income Taxes (calculated)' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns:
            income_statement['Net Income - Net Income (calculated)'] = income_statement['Income Before Tax - Income Before Income Taxes (calculated)'] - income_statement['Tax - Income Tax Expense/Benefit']
    
    # Add validation checks for key relationships
    # Validation: Revenue - COGS = Gross Profit
    if 'Revenue - Total Revenue' in income_statement.columns and 'Cost - Total Cost of Revenue' in income_statement.columns and 'Gross - Gross Profit' in income_statement.columns:
        income_statement['Validation - Gross Profit Difference'] = income_statement['Gross - Gross Profit'] - (income_statement['Revenue - Total Revenue'] - income_statement['Cost - Total Cost of Revenue'])
    
    # Validation: Gross Profit - Operating Expenses = Operating Income
    if 'Gross - Gross Profit' in income_statement.columns and 'Operating Expenses - Total Operating Expenses' in income_statement.columns and 'Operating - Operating Income' in income_statement.columns:
        income_statement['Validation - Operating Income Difference'] = income_statement['Operating - Operating Income'] - (income_statement['Gross - Gross Profit'] - income_statement['Operating Expenses - Total Operating Expenses'])
    
    # Validation: Income Before Tax - Income Tax = Net Income
    if 'Income Before Tax - Income Before Income Taxes' in income_statement.columns and 'Tax - Income Tax Expense/Benefit' in income_statement.columns and 'Net Income - Net Income' in income_statement.columns:
        income_statement['Validation - Net Income Difference'] = income_statement['Net Income - Net Income'] - (income_statement['Income Before Tax - Income Before Income Taxes'] - income_statement['Tax - Income Tax Expense/Benefit'])
    
    return income_statement

# Function to display a human-readable income statement
def display_income_statement(income_statement, in_millions=True):
    """
    Display the income statement in a clean, readable format.
    
    Args:
        income_statement: DataFrame with the reconstructed income statement
        in_millions: If True, display values in millions; otherwise in thousands
    
    Returns:
        DataFrame with the formatted income statement
    """
    # Make a copy to avoid modifying the original
    formatted_is = income_statement.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_is.columns if any(col.startswith(prefix) for prefix in ['Revenue', 'Cost', 'Gross', 'Operating', 'Other', 'Income Before Tax', 'Tax', 'Net Income', 'EPS', 'Validation'])]
    
    # Convert to millions or thousands
    divisor = 1_000_000 if in_millions else 1_000
    suffix = 'M' if in_millions else 'K'
    
    for col in numeric_cols:
        if 'EPS' not in col:  # Don't scale EPS values
            formatted_is[col] = formatted_is[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_is.columns:
        try:
            formatted_is['end'] = pd.to_datetime(formatted_is['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display in proper income statement order
    display_sections = [
        'Revenue',
        'Cost',
        'Gross',
        'Operating Expenses',
        'Operating',
        'Other',
        'Income Before Tax',
        'Tax',
        'Net Income',
        'EPS'
    ]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_is.columns else []
    for section in display_sections:
        section_cols = [col for col in formatted_is.columns if col.startswith(f"{section} - ")]
        display_cols.extend(section_cols)
    
    # Add validation columns at the end if they exist
    validation_cols = [col for col in formatted_is.columns if col.startswith('Validation - ')]
    display_cols.extend(validation_cols)
    
    return formatted_is[display_cols]

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)

# Example usage
def analyze_microsoft_income_statement(df):
    """
    Analyze Microsoft's income statement from XBRL data.
    
    Args:
        df: DataFrame containing Microsoft's XBRL-tagged financial data
    
    Returns:
        The reconstructed income statement
    """
    print("Reconstructing Microsoft's income statement from XBRL data...")
    income_statement = reconstruct_income_statement(df)
    
    # Remove columns with only NaN values
    income_statement = remove_nan_only_columns(income_statement)
    
    print("\nIncome Statement Summary:")
    print(f"- Total columns: {len(income_statement.columns)}")
    print(f"- Data for {len(income_statement)} reporting periods")
    
    # Display the income statement
    print("\nFormatted Income Statement (in millions USD):")
    formatted_is = display_income_statement(income_statement, in_millions=True)
    
    return income_statement, formatted_is

# To use this code with your DataFrame:
income_statement, formatted_is = analyze_microsoft_income_statement(consolidated_df)

Reconstructing Microsoft's income statement from XBRL data...

Income Statement Summary:
- Total columns: 27
- Data for 29 reporting periods

Formatted Income Statement (in millions USD):


In [271]:
consolidated_df.to_csv('./test.csv')

In [272]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
income_statement = income_statement[~income_statement['frame'].str.match(r'CY\d{4}Q[1-4].+')]
income_statement = income_statement.sort_values(by='end')
income_statement.T

,52,53,54,57,58,59,61,62,63,64,...,77,78,80,82,83,84,85,87,88,89
filed,2019-08-01,2019-08-01,2019-08-01,2020-07-31,2020-07-31,2020-07-31,2019-08-01,2021-07-29,2021-07-29,2021-07-29,...,2024-12-03,2023-07-27,2024-12-03,2023-07-27,2023-10-24,2024-01-30,2024-12-03,2024-12-03,2025-01-29,2025-01-29
company_name,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,...,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
end,2017-09-30,2017-12-31,2018-03-31,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,...,8-K,10-K,8-K,10-K,10-Q,10-Q,8-K,8-K,10-Q,10-Q
frame,CY2017Q3,CY2017Q4,CY2018Q1,CY2018Q3,CY2018Q4,CY2019Q1,CY2019,CY2019Q3,CY2019Q4,CY2020Q1,...,CY2022Q3,CY2022Q4,CY2023Q1,CY2023,CY2023Q3,CY2023Q4,CY2024Q1,CY2024,CY2024Q3,CY2024Q4
cik,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019,...,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019
Revenue - Total Revenue,24538000000.0,28918000000.0,26819000000.0,29084000000.0,32471000000.0,30571000000.0,125843000000.0,33055000000.0,36906000000.0,35021000000.0,...,50122000000.0,52747000000.0,52857000000.0,211915000000.0,56517000000.0,62020000000.0,61858000000.0,245122000000.0,65585000000.0,69632000000.0
Cost - Total Cost of Revenue,8278000000.0,11064000000.0,9269000000.0,9905000000.0,12423000000.0,10170000000.0,42910000000.0,10406000000.0,12358000000.0,10975000000.0,...,15452000000.0,17488000000.0,16128000000.0,65863000000.0,16302000000.0,19623000000.0,18505000000.0,74114000000.0,20099000000.0,21799000000.0
Gross - Gross Profit,16260000000.0,17854000000.0,17550000000.0,19179000000.0,20048000000.0,20401000000.0,82933000000.0,22649000000.0,24548000000.0,24046000000.0,...,34670000000.0,35259000000.0,36729000000.0,146052000000.0,40215000000.0,42397000000.0,43353000000.0,171008000000.0,45486000000.0,47833000000.0


## CashFlow Statement

In [273]:
import pandas as pd
import numpy as np

def reconstruct_cash_flow_statement(df):
    """
    Reconstructs cash flow statement from XBRL-tagged data, specifically tailored for IT companies like Microsoft.
    
    Args:
        df: DataFrame containing XBRL-tagged financial data
        
    Returns:
        DataFrame with reconstructed cash flow statement for each period
    """
    # Create a new DataFrame to store the reconstructed cash flow statement
    cash_flow = pd.DataFrame(index=df.index)
    
    # Copy identifying columns
    id_columns = ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik']
    for col in id_columns:
        if col in df.columns:
            cash_flow[col] = df[col]
    
    # Get all available columns for matching
    available_columns = set(df.columns)
    
    # Define mappings for cash flow statement accounts based on XBRL structure
    
    # Operating Activities section
    operating_activities = {
        'Net Income': {
            'primary': 'ProfitLoss',
            'alternatives': ['NetIncomeLoss'],
            'children': []
        },
        'Depreciation and Amortization': {
            'primary': 'DepreciationAndAmortization',
            'alternatives': [],
            'children': ['Depreciation', 'AmortizationOfIntangibleAssets', 'AmortizationOfCapitalizedSoftwareCosts']
        },
        'Stock-based Compensation': {
            'primary': 'StockBasedCompensationExpense',
            'alternatives': ['ShareBasedCompensation'],
            'children': []
        },
        'Deferred Income Taxes': {
            'primary': 'DeferredIncomeTaxExpenseBenefit',
            'alternatives': ['IncomeTaxExpenseBenefitDeferredTax'],
            'children': []
        },
        'Impairment of Goodwill': {
            'primary': 'ImpairmentOfGoodwill',
            'alternatives': [],
            'children': []
        },
        'Impairment of Intangible Assets': {
            'primary': 'ImpairmentOfIntangibleAssets',
            'alternatives': [],
            'children': []
        },
        'Net Gains/Losses on Investments': {
            'primary': 'GainLossOnInvestments',
            'alternatives': ['InvestmentIncomeNetGainLossFromSaleOfInvestment'],
            'children': []
        },
        'Net Gains/Losses on Asset Sales': {
            'primary': 'GainLossOnSaleOfPropertyPlantAndEquipment',
            'alternatives': [],
            'children': []
        },
        'Changes in Operating Assets and Liabilities': {
            'primary': 'ChangeInOperatingAssetsAndLiabilities',
            'alternatives': [],
            'children': []
        },
        'Change in Accounts Receivable': {
            'primary': 'ChangeInAccountsReceivable',
            'alternatives': [],
            'children': []
        },
        'Change in Inventory': {
            'primary': 'ChangeInInventory',
            'alternatives': [],
            'children': []
        },
        'Change in Prepaid Expenses and Other Assets': {
            'primary': 'ChangeInPrepaidExpensesAndOtherAssets',
            'alternatives': [],
            'children': ['ChangeInPrepaidExpenses', 'ChangeInOtherAssets']
        },
        'Change in Accounts Payable': {
            'primary': 'ChangeInAccountsPayable',
            'alternatives': [],
            'children': []
        },
        'Change in Accrued Liabilities': {
            'primary': 'ChangeInAccruedLiabilities',
            'alternatives': ['ChangeInAccruedExpenses'],
            'children': ['ChangeInAccruedCompensation']
        },
        'Change in Deferred Revenue': {
            'primary': 'ChangeInDeferredRevenue',
            'alternatives': ['ChangeInContractLiabilities', 'ChangeInUnearnedRevenue'],
            'children': []
        },
        'Change in Income Taxes': {
            'primary': 'ChangeInIncomeTaxesPayableReceivable',
            'alternatives': ['ChangeInIncomeTaxPayable'],
            'children': []
        },
        'Other Operating Activities': {
            'primary': 'OtherOperatingActivitiesCashFlowAdjustments',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Operating Activities': {
            'primary': 'NetCashProvidedByUsedInOperatingActivities',
            'alternatives': ['CashFlowsFromUsedInOperatingActivities'],
            'children': []
        }
    }
    
    # Investing Activities section
    investing_activities = {
        'Capital Expenditures': {
            'primary': 'PaymentsToAcquirePropertyPlantAndEquipment',
            'alternatives': ['CapitalExpenditures'],
            'children': []
        },
        'Proceeds from Asset Sales': {
            'primary': 'ProceedsFromSaleOfPropertyPlantAndEquipment',
            'alternatives': [],
            'children': []
        },
        'Purchases of Intangible Assets': {
            'primary': 'PaymentsToAcquireIntangibleAssetsExcludingGoodwill',
            'alternatives': [],
            'children': []
        },
        'Acquisitions, Net of Cash Acquired': {
            'primary': 'PaymentsToAcquireBusinessesNetOfCashAcquired',
            'alternatives': ['BusinessAcquisitionsNetCashAcquired'],
            'children': []
        },
        'Purchases of Investments': {
            'primary': 'PaymentsToAcquireInvestments',
            'alternatives': ['PurchaseOfInvestments'],
            'children': []
        },
        'Proceeds from Sale of Investments': {
            'primary': 'ProceedsFromSaleOfInvestments',
            'alternatives': [],
            'children': []
        },
        'Proceeds from Maturities of Investments': {
            'primary': 'ProceedsFromMaturitiesOfInvestments',
            'alternatives': ['ProceedsFromMaturityOfInvestments'],
            'children': []
        },
        'Other Investing Activities': {
            'primary': 'OtherInvestingActivities',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Investing Activities': {
            'primary': 'NetCashProvidedByUsedInInvestingActivities',
            'alternatives': ['CashFlowsFromUsedInInvestingActivities'],
            'children': []
        }
    }
    
    # Financing Activities section
    financing_activities = {
        'Proceeds from Long-term Debt': {
            'primary': 'ProceedsFromIssuanceOfLongTermDebt',
            'alternatives': ['ProceedsFromDebtIssuance'],
            'children': []
        },
        'Repayment of Long-term Debt': {
            'primary': 'PaymentsOfPrincipalOnLongTermDebt',
            'alternatives': ['RepaymentsOfLongTermDebt', 'DebtRepayment'],
            'children': []
        },
        'Proceeds from Common Stock Issuance': {
            'primary': 'ProceedsFromIssuanceOfCommonStock',
            'alternatives': ['ProceedsFromStockPlans'],
            'children': []
        },
        'Common Stock Repurchases': {
            'primary': 'PaymentsForRepurchaseOfCommonStock',
            'alternatives': ['PaymentsToRepurchaseCommonStock', 'StockRepurchaseProgram'],
            'children': []
        },
        'Dividend Payments': {
            'primary': 'PaymentsOfDividends',
            'alternatives': ['DividendsPaid'],
            'children': []
        },
        'Lease Payments': {
            'primary': 'PaymentsOfFinanceLeaseLiabilities',
            'alternatives': ['PaymentsOfCapitalLeaseObligations'],
            'children': []
        },
        'Other Financing Activities': {
            'primary': 'OtherFinancingActivities',
            'alternatives': [],
            'children': []
        },
        'Net Cash from Financing Activities': {
            'primary': 'NetCashProvidedByUsedInFinancingActivities',
            'alternatives': ['CashFlowsFromUsedInFinancingActivities'],
            'children': []
        }
    }
    
    # Effect of Exchange Rate Changes and Net Change in Cash
    cash_reconciliation = {
        'Effect of Exchange Rate Changes': {
            'primary': 'EffectOfExchangeRateOnCashAndCashEquivalents',
            'alternatives': ['EffectOfExchangeRateChangesOnCash'],
            'children': []
        },
        'Net Change in Cash and Cash Equivalents': {
            'primary': 'IncreaseDecreaseInCashAndCashEquivalents',
            'alternatives': ['ChangeInCashAndCashEquivalents', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecrease'],
            'children': []
        },
        'Cash and Cash Equivalents, Beginning of Period': {
            'primary': 'CashAndCashEquivalentsAtCarryingValue',  # This will be filtered by context to beginning period
            'alternatives': ['CashAndCashEquivalentsPeriodStart', 'CashPeriodStart'],
            'children': []
        },
        'Cash and Cash Equivalents, End of Period': {
            'primary': 'CashAndCashEquivalentsAtCarryingValue',  # This will be filtered by context to ending period
            'alternatives': ['CashAndCashEquivalentsPeriodEnd', 'CashPeriodEnd'],
            'children': []
        }
    }
    
    # Supplemental information
    supplemental_info = {
        'Interest Paid': {
            'primary': 'InterestPaid',
            'alternatives': [],
            'children': []
        },
        'Income Taxes Paid': {
            'primary': 'IncomeTaxesPaid',
            'alternatives': [],
            'children': []
        }
    }
    
    # Helper function to find the best matching column
    def find_best_match(account_info, available_cols):
        # Check primary tag
        if account_info['primary'] in available_cols:
            return account_info['primary']
        
        # Check alternatives
        for alt in account_info['alternatives']:
            if alt in available_cols:
                return alt
                
        # Handle partial matches for more flexible matching
        primary_matches = []
        for col in available_cols:
            if account_info['primary'] in col:
                primary_matches.append(col)
        
        # If we found exactly one partial match for primary, return it
        if len(primary_matches) == 1:
            return primary_matches[0]
                
        # If we found multiple partial matches, try to find the best one
        if primary_matches:
            for match in primary_matches:
                # For cash flow statement, look for "CashFlow" or "NetCash" in the match
                if 'CashFlow' in match or 'NetCash' in match:
                    return match
            
            return primary_matches[0]  # Default to first one
        
        # Check for partial matches in alternatives
        for alt in account_info['alternatives']:
            for col in available_cols:
                if alt in col:
                    return col
        
        return None
    
    # Process each section
    def process_section(section_name, section_mapping):
        for account_name, account_info in section_mapping.items():
            # Special handling for Cash and Cash Equivalents at beginning/end of period
            if account_name == 'Cash and Cash Equivalents, Beginning of Period':
                # Look for beginning period cash
                for col in available_columns:
                    if 'CashAndCashEquivalents' in col and 'Begin' in col:
                        cash_flow[f"{section_name} - {account_name}"] = df[col]
                        break
                else:
                    # Try to find regular cash column with beginning period context
                    best_match = find_best_match(account_info, available_columns)
                    if best_match:
                        cash_flow[f"{section_name} - {account_name}"] = df[best_match]
                    else:
                        cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
                continue
                
            if account_name == 'Cash and Cash Equivalents, End of Period':
                # Look for ending period cash
                for col in available_columns:
                    if 'CashAndCashEquivalents' in col and 'End' in col:
                        cash_flow[f"{section_name} - {account_name}"] = df[col]
                        break
                else:
                    # Try to find regular cash column with ending period context
                    best_match = find_best_match(account_info, available_columns)
                    if best_match:
                        cash_flow[f"{section_name} - {account_name}"] = df[best_match]
                    else:
                        cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
                continue
            
            # Standard processing for other accounts
            best_match = find_best_match(account_info, available_columns)
            
            if best_match:
                # Direct match found
                cash_flow[f"{section_name} - {account_name}"] = df[best_match]
            elif account_info['children']:
                # Try to sum children if parent not found
                available_children = []
                for child in account_info['children']:
                    if child in available_columns:
                        available_children.append(child)
                    else:
                        # Look for variations
                        for col in available_columns:
                            if child in col:
                                available_children.append(col)
                
                if available_children:
                    cash_flow[f"{section_name} - {account_name}"] = df[available_children].sum(axis=1)
                else:
                    # No matches found
                    cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
            else:
                # No matches found
                cash_flow[f"{section_name} - {account_name}"] = pd.Series(np.nan, index=df.index)
    
    # Process each section
    process_section("Operating", operating_activities)
    process_section("Investing", investing_activities)
    process_section("Financing", financing_activities)
    process_section("Cash", cash_reconciliation)
    process_section("Supplemental", supplemental_info)
    
    # Calculate missing values and add validation checks
    
    # If Net Cash from Operating Activities is missing, try to calculate it
    if ('Operating - Net Cash from Operating Activities' not in cash_flow.columns or 
        cash_flow['Operating - Net Cash from Operating Activities'].isna().all()):
        
        operating_cols = [col for col in cash_flow.columns 
                          if col.startswith('Operating - ') and 'Net Cash' not in col]
        
        if operating_cols and 'Operating - Net Income' in operating_cols:
            # Start with net income
            cash_flow['Operating - Net Cash from Operating Activities (calculated)'] = cash_flow['Operating - Net Income']
            
            # Add all other operating activities except net income
            for col in operating_cols:
                if col != 'Operating - Net Income':
                    # Skip NaN columns
                    if not cash_flow[col].isna().all():
                        cash_flow['Operating - Net Cash from Operating Activities (calculated)'] += cash_flow[col]
    
    # If Net Cash from Investing Activities is missing, try to calculate it
    if ('Investing - Net Cash from Investing Activities' not in cash_flow.columns or 
        cash_flow['Investing - Net Cash from Investing Activities'].isna().all()):
        
        investing_cols = [col for col in cash_flow.columns 
                          if col.startswith('Investing - ') and 'Net Cash' not in col]
        
        if investing_cols:
            cash_flow['Investing - Net Cash from Investing Activities (calculated)'] = 0
            
            # Sum all investing activities
            for col in investing_cols:
                # Skip NaN columns
                if not cash_flow[col].isna().all():
                    cash_flow['Investing - Net Cash from Investing Activities (calculated)'] += cash_flow[col]
    
    # If Net Cash from Financing Activities is missing, try to calculate it
    if ('Financing - Net Cash from Financing Activities' not in cash_flow.columns or 
        cash_flow['Financing - Net Cash from Financing Activities'].isna().all()):
        
        financing_cols = [col for col in cash_flow.columns 
                          if col.startswith('Financing - ') and 'Net Cash' not in col]
        
        if financing_cols:
            cash_flow['Financing - Net Cash from Financing Activities (calculated)'] = 0
            
            # Sum all financing activities
            for col in financing_cols:
                # Skip NaN columns
                if not cash_flow[col].isna().all():
                    cash_flow['Financing - Net Cash from Financing Activities (calculated)'] += cash_flow[col]
    
    # If Net Change in Cash is missing, calculate it from operating, investing, and financing
    if ('Cash - Net Change in Cash and Cash Equivalents' not in cash_flow.columns or 
        cash_flow['Cash - Net Change in Cash and Cash Equivalents'].isna().all()):
        
        # Choose the best available values for each section
        op_cash = ('Operating - Net Cash from Operating Activities' 
                   if 'Operating - Net Cash from Operating Activities' in cash_flow.columns 
                   else 'Operating - Net Cash from Operating Activities (calculated)')
        
        inv_cash = ('Investing - Net Cash from Investing Activities' 
                    if 'Investing - Net Cash from Investing Activities' in cash_flow.columns 
                    else 'Investing - Net Cash from Investing Activities (calculated)')
        
        fin_cash = ('Financing - Net Cash from Financing Activities' 
                    if 'Financing - Net Cash from Financing Activities' in cash_flow.columns 
                    else 'Financing - Net Cash from Financing Activities (calculated)')
        
        fx_effect = 'Cash - Effect of Exchange Rate Changes'
        
        # Calculate net change in cash if we have at least operating, investing, and financing
        cash_components = []
        for component in [op_cash, inv_cash, fin_cash]:
            if component in cash_flow.columns and not cash_flow[component].isna().all():
                cash_components.append(component)
                
        # Add forex effect if available
        if fx_effect in cash_flow.columns and not cash_flow[fx_effect].isna().all():
            cash_components.append(fx_effect)
            
        if len(cash_components) >= 3:  # At least have operating, investing, and financing
            cash_flow['Cash - Net Change in Cash and Cash Equivalents (calculated)'] = cash_flow[cash_components].sum(axis=1)
    
    # If Beginning Cash + Net Change doesn't equal Ending Cash, add validation check
    if ('Cash - Cash and Cash Equivalents, Beginning of Period' in cash_flow.columns and 
        'Cash - Cash and Cash Equivalents, End of Period' in cash_flow.columns):
        
        # Choose best available net change value
        net_change_col = ('Cash - Net Change in Cash and Cash Equivalents' 
                           if 'Cash - Net Change in Cash and Cash Equivalents' in cash_flow.columns 
                           else 'Cash - Net Change in Cash and Cash Equivalents (calculated)')
        
        if net_change_col in cash_flow.columns:
            cash_flow['Validation - Cash Reconciliation Difference'] = (
                cash_flow['Cash - Cash and Cash Equivalents, End of Period'] - 
                (cash_flow['Cash - Cash and Cash Equivalents, Beginning of Period'] + cash_flow[net_change_col])
            )
    
    # Add validation for operating + investing + financing = net change
    op_cash_col = next((col for col in cash_flow.columns 
                        if col.startswith('Operating - Net Cash') and not cash_flow[col].isna().all()), None)
    
    inv_cash_col = next((col for col in cash_flow.columns 
                         if col.startswith('Investing - Net Cash') and not cash_flow[col].isna().all()), None)
    
    fin_cash_col = next((col for col in cash_flow.columns 
                         if col.startswith('Financing - Net Cash') and not cash_flow[col].isna().all()), None)
    
    net_change_col = next((col for col in cash_flow.columns 
                           if col.startswith('Cash - Net Change') and not cash_flow[col].isna().all()), None)
    
    fx_effect_col = next((col for col in cash_flow.columns 
                          if 'Effect of Exchange Rate' in col and not cash_flow[col].isna().all()), None)
    
    if op_cash_col and inv_cash_col and fin_cash_col and net_change_col:
        components_sum = cash_flow[op_cash_col] + cash_flow[inv_cash_col] + cash_flow[fin_cash_col]
        
        # Add forex effect if available
        if fx_effect_col:
            components_sum += cash_flow[fx_effect_col]
            
        cash_flow['Validation - Net Cash Components Difference'] = cash_flow[net_change_col] - components_sum
    
    return cash_flow

# Function to display a human-readable cash flow statement
def display_cash_flow_statement(cash_flow, in_millions=True):
    """
    Display the cash flow statement in a clean, readable format.
    
    Args:
        cash_flow: DataFrame with the reconstructed cash flow statement
        in_millions: If True, display values in millions; otherwise in thousands
    
    Returns:
        DataFrame with the formatted cash flow statement
    """
    # Make a copy to avoid modifying the original
    formatted_cf = cash_flow.copy()
    
    # Identify numeric columns
    numeric_cols = [col for col in formatted_cf.columns 
                    if any(col.startswith(prefix) for prefix in ['Operating', 'Investing', 'Financing', 'Cash', 'Supplemental', 'Validation'])]
    
    # Convert to millions or thousands
    divisor = 1_000_000 if in_millions else 1_000
    suffix = 'M' if in_millions else 'K'
    
    for col in numeric_cols:
        formatted_cf[col] = formatted_cf[col] / divisor
        
    # Format the date column if it exists
    if 'end' in formatted_cf.columns:
        try:
            formatted_cf['end'] = pd.to_datetime(formatted_cf['end']).dt.strftime('%Y-%m-%d')
        except:
            pass
    
    # Create sections for display in proper cash flow statement order
    display_sections = [
        'Operating',
        'Investing',
        'Financing',
        'Cash',
        'Supplemental'
    ]
    
    # Columns to display
    display_cols = ['end'] if 'end' in formatted_cf.columns else []
    for section in display_sections:
        section_cols = [col for col in formatted_cf.columns if col.startswith(f"{section} - ")]
        display_cols.extend(section_cols)
    
    # Add validation columns at the end if they exist
    validation_cols = [col for col in formatted_cf.columns if col.startswith('Validation - ')]
    display_cols.extend(validation_cols)
    
    return formatted_cf[display_cols]

# Function to remove columns that only contain NaN values
def remove_nan_only_columns(df):
    """
    Removes columns that contain only NaN values.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        DataFrame with NaN-only columns removed
    """
    # Find columns where all values are NaN
    nan_cols = df.columns[df.isna().all()].tolist()
    
    # Drop those columns
    return df.drop(columns=nan_cols)

# Example usage
def analyze_microsoft_cash_flow(df):
    """
    Analyze Microsoft's cash flow statement from XBRL data.
    
    Args:
        df: DataFrame containing Microsoft's XBRL-tagged financial data
    
    Returns:
        The reconstructed cash flow statement
    """
    print("Reconstructing Microsoft's cash flow statement from XBRL data...")
    cash_flow = reconstruct_cash_flow_statement(df)
    
    # Remove columns with only NaN values
    cash_flow = remove_nan_only_columns(cash_flow)
    
    print("\nCash Flow Statement Summary:")
    print(f"- Total columns: {len(cash_flow.columns)}")
    print(f"- Data for {len(cash_flow)} reporting periods")
    
    # Display the cash flow statement
    print("\nFormatted Cash Flow Statement (in millions USD):")
    formatted_cf = display_cash_flow_statement(cash_flow, in_millions=True)
    
    return cash_flow, formatted_cf

# To use this code with your DataFrame:
cash_flow, formatted_cf = analyze_microsoft_cash_flow(consolidated_df)

Reconstructing Microsoft's cash flow statement from XBRL data...

Cash Flow Statement Summary:
- Total columns: 30
- Data for 29 reporting periods

Formatted Cash Flow Statement (in millions USD):


In [274]:
# Keep rows with exactly Q1, Q2, Q3, Q4 or just the year (like CY2019)
cash_flow = cash_flow[~cash_flow['frame'].str.match(r'CY\d{4}Q[1-4].+')]
cash_flow = cash_flow.sort_values(by='end')
cash_flow.T

,52,53,54,57,58,59,61,62,63,64,...,77,78,80,82,83,84,85,87,88,89
filed,2019-08-01,2019-08-01,2019-08-01,2020-07-31,2020-07-31,2020-07-31,2019-08-01,2021-07-29,2021-07-29,2021-07-29,...,2024-12-03,2023-07-27,2024-12-03,2023-07-27,2023-10-24,2024-01-30,2024-12-03,2024-12-03,2025-01-29,2025-01-29
company_name,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,...,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
end,2017-09-30,2017-12-31,2018-03-31,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,...,8-K,10-K,8-K,10-K,10-Q,10-Q,8-K,8-K,10-Q,10-Q
frame,CY2017Q3,CY2017Q4,CY2018Q1,CY2018Q3,CY2018Q4,CY2019Q1,CY2019,CY2019Q3,CY2019Q4,CY2020Q1,...,CY2022Q3,CY2022Q4,CY2023Q1,CY2023,CY2023Q3,CY2023Q4,CY2024Q1,CY2024,CY2024Q3,CY2024Q4
cik,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019,...,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019
Operating - Net Income,6576000000.0,-6302000000.0,7424000000.0,8824000000.0,8420000000.0,8809000000.0,39240000000.0,10678000000.0,11649000000.0,10752000000.0,...,17556000000.0,16425000000.0,18299000000.0,72361000000.0,22291000000.0,21870000000.0,21939000000.0,88136000000.0,24667000000.0,24108000000.0
Operating - Depreciation and Amortization,563000000.0,562000000.0,560000000.0,556000000.0,530000000.0,431000000.0,11600000000.0,434000000.0,411000000.0,356000000.0,...,633000000.0,632000000.0,612000000.0,13500000000.0,636000000.0,1300000000.0,1400000000.0,20000000000.0,1400000000.0,1500000000.0
Operating - Stock-based Compensation,973000000.0,986000000.0,969000000.0,1107000000.0,1183000000.0,1172000000.0,4652000000.0,1262000000.0,1340000000.0,1338000000.0,...,2192000000.0,2538000000.0,2465000000.0,9611000000.0,2507000000.0,2828000000.0,2703000000.0,10734000000.0,2832000000.0,3089000000.0


In [275]:
balance_sheet.head(10)

,filed,company_name,end,unit,form,frame,cik,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Liabilities - Other Long-term Liabilities,Liabilities - Total Liabilities,Equity - Common Stock and Paid-in Capital,Equity - Retained Earnings,Equity - Accumulated Other Comprehensive Income,Equity - Total Stockholders Equity,Equity - Total Liabilities and Equity,Validation - Current Assets Difference,Validation - Current Liabilities Difference,Validation - A = L+E Difference
52,2019-08-01,MICROSOFT CORPORATION,2017-09-30,USD,10-K,CY2017Q3,789019,6.884000e+09,NaN,1.456100e+10,...,4.229000e+09,1.594500e+11,6.941900e+10,1.970200e+10,5.260000e+08,8.964700e+10,2.490970e+11,1.416820e+11,4.304500e+10,0.0
53,2019-08-01,MICROSOFT CORPORATION,2017-12-31,USD,10-K,CY2017Q4,789019,1.285900e+10,NaN,1.842800e+10,...,4.820000e+09,1.776430e+11,7.019200e+10,8.567000e+09,-3.990000e+08,7.836000e+10,2.560030e+11,1.447830e+11,4.686600e+10,0.0
54,2019-08-01,MICROSOFT CORPORATION,2018-03-31,USD,10-K,CY2018Q1,789019,9.221000e+09,NaN,1.720800e+10,...,5.223000e+09,1.662580e+11,7.041800e+10,9.974000e+09,-1.153000e+09,7.923900e+10,2.454970e+11,1.343540e+11,3.518600e+10,0.0
57,2020-07-31,MICROSOFT CORPORATION,2018-09-30,USD,10-K,CY2018Q3,789019,1.513700e+10,1.207430e+11,1.739000e+10,...,5.539000e+09,1.716520e+11,7.130300e+10,1.727900e+10,-2.615000e+09,8.596700e+10,2.576190e+11,1.394940e+11,4.182700e+10,0.0
58,2020-07-31,MICROSOFT CORPORATION,2018-12-31,USD,10-K,CY2018Q4,789019,6.638000e+09,1.210240e+11,1.968000e+10,...,6.055000e+09,1.667310e+11,7.755600e+10,1.658500e+10,-2.013000e+09,9.212800e+10,2.588590e+11,1.296230e+11,3.850500e+10,0.0
59,2020-07-31,MICROSOFT CORPORATION,2019-03-31,USD,10-K,CY2019Q1,789019,1.121200e+10,1.204060e+11,1.926900e+10,...,6.763000e+09,1.684170e+11,7.779100e+10,1.833800e+10,-1.265000e+09,9.486400e+10,2.632810e+11,1.335690e+11,3.950900e+10,0.0
61,2019-08-01,MICROSOFT CORPORATION,2019-06-30,USD,10-K,CY2019,789019,1.135600e+10,1.224630e+11,2.952400e+10,...,7.581000e+09,1.842260e+11,7.852000e+10,2.415000e+10,-3.400000e+08,1.023300e+11,2.865560e+11,1.358820e+11,5.455300e+10,0.0
62,2021-07-29,MICROSOFT CORPORATION,2019-09-30,USD,10-K,CY2019Q3,789019,1.311700e+10,1.235190e+11,1.908700e+10,...,8.826000e+09,1.728940e+11,7.888200e+10,2.724000e+10,-6.100000e+07,1.060610e+11,2.789550e+11,1.392580e+11,4.659400e+10,0.0
63,2021-07-29,MICROSOFT CORPORATION,2019-12-31,USD,10-K,CY2019Q4,789019,8.864000e+09,1.253890e+11,2.352500e+10,...,9.658000e+09,1.726850e+11,7.962500e+10,3.073900e+10,-2.550000e+08,1.101090e+11,2.827940e+11,1.360760e+11,4.426200e+10,0.0
64,2021-07-29,MICROSOFT CORPORATION,2020-03-31,USD,10-K,CY2020Q1,789019,1.171000e+10,1.259160e+11,2.269900e+10,...,9.673000e+09,1.709480e+11,7.981300e+10,3.201200e+10,2.676000e+09,1.145010e+11,2.854490e+11,1.392700e+11,4.580800e+10,0.0


In [276]:
balance_sheet.to_csv('./MSFT_bs.csv')
income_statement.to_csv('./MSFT_is.csv')

In [277]:
income_statement.head(10)

,filed,company_name,end,unit,form,frame,cik,Revenue - Total Revenue,Cost - Total Cost of Revenue,Gross - Gross Profit,...,Other - Interest Expense,Other - Gain/Loss on Investments,Other - Foreign Currency Transaction Gain/Loss,"Other - Other Income/Expense, Net",Tax - Income Tax Expense/Benefit,Net Income - Net Income,EPS - Basic EPS,EPS - Diluted EPS,Income Before Tax - Income Before Income Taxes (calculated),Validation - Gross Profit Difference
52,2019-08-01,MICROSOFT CORPORATION,2017-09-30,USD,10-K,CY2017Q3,789019,2.453800e+10,8.278000e+09,1.626000e+10,...,6.720000e+08,573000000.0,-9000000.0,276000000.0,1.408000e+09,6.576000e+09,0.85,0.84,7.984000e+09,0.0
53,2019-08-01,MICROSOFT CORPORATION,2017-12-31,USD,10-K,CY2017Q4,789019,2.891800e+10,1.106400e+10,1.785400e+10,...,6.980000e+08,768000000.0,-60000000.0,490000000.0,1.547100e+10,-6.302000e+09,-0.82,-0.82,9.169000e+09,0.0
54,2019-08-01,MICROSOFT CORPORATION,2018-03-31,USD,10-K,CY2018Q1,789019,2.681900e+10,9.269000e+09,1.755000e+10,...,6.910000e+08,510000000.0,20000000.0,349000000.0,1.217000e+09,7.424000e+09,0.96,0.95,8.641000e+09,0.0
57,2020-07-31,MICROSOFT CORPORATION,2018-09-30,USD,10-K,CY2018Q3,789019,2.908400e+10,9.905000e+09,1.917900e+10,...,6.740000e+08,243000000.0,5000000.0,266000000.0,1.397000e+09,8.824000e+09,1.15,1.14,1.022100e+10,0.0
58,2020-07-31,MICROSOFT CORPORATION,2018-12-31,USD,10-K,CY2018Q4,789019,3.247100e+10,1.242300e+10,2.004800e+10,...,6.720000e+08,94000000.0,-74000000.0,127000000.0,1.965000e+09,8.420000e+09,1.09,1.08,1.038500e+10,0.0
59,2020-07-31,MICROSOFT CORPORATION,2019-03-31,USD,10-K,CY2019Q1,789019,3.057100e+10,1.017000e+10,2.040100e+10,...,6.710000e+08,44000000.0,37000000.0,145000000.0,1.677000e+09,8.809000e+09,1.15,1.14,1.048600e+10,0.0
61,2019-08-01,MICROSOFT CORPORATION,2019-06-30,USD,10-K,CY2019,789019,1.258430e+11,4.291000e+10,8.293300e+10,...,2.686000e+09,648000000.0,-82000000.0,729000000.0,4.448000e+09,3.924000e+10,5.11,5.06,4.368800e+10,0.0
62,2021-07-29,MICROSOFT CORPORATION,2019-09-30,USD,10-K,CY2019Q3,789019,3.305500e+10,1.040600e+10,2.264900e+10,...,6.370000e+08,-57000000.0,-58000000.0,0.0,2.008000e+09,1.067800e+10,1.40,1.38,1.268600e+10,0.0
63,2021-07-29,MICROSOFT CORPORATION,2019-12-31,USD,10-K,CY2019Q4,789019,3.690600e+10,1.235800e+10,2.454800e+10,...,6.540000e+08,162000000.0,-24000000.0,194000000.0,2.436000e+09,1.164900e+10,1.53,1.51,1.408500e+10,0.0
64,2021-07-29,MICROSOFT CORPORATION,2020-03-31,USD,10-K,CY2020Q1,789019,3.502100e+10,1.097500e+10,2.404600e+10,...,6.140000e+08,-101000000.0,-136000000.0,-132000000.0,2.091000e+09,1.075200e+10,1.41,1.40,1.284300e+10,0.0


In [278]:
cash_flow.T

,52,53,54,57,58,59,61,62,63,64,...,77,78,80,82,83,84,85,87,88,89
filed,2019-08-01,2019-08-01,2019-08-01,2020-07-31,2020-07-31,2020-07-31,2019-08-01,2021-07-29,2021-07-29,2021-07-29,...,2024-12-03,2023-07-27,2024-12-03,2023-07-27,2023-10-24,2024-01-30,2024-12-03,2024-12-03,2025-01-29,2025-01-29
company_name,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,...,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
end,2017-09-30,2017-12-31,2018-03-31,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,...,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31,2024-06-30,2024-09-30,2024-12-31
unit,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
form,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,10-K,...,8-K,10-K,8-K,10-K,10-Q,10-Q,8-K,8-K,10-Q,10-Q
frame,CY2017Q3,CY2017Q4,CY2018Q1,CY2018Q3,CY2018Q4,CY2019Q1,CY2019,CY2019Q3,CY2019Q4,CY2020Q1,...,CY2022Q3,CY2022Q4,CY2023Q1,CY2023,CY2023Q3,CY2023Q4,CY2024Q1,CY2024,CY2024Q3,CY2024Q4
cik,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019,...,789019,789019,789019,789019,789019,789019,789019,789019,789019,789019
Operating - Net Income,6576000000.0,-6302000000.0,7424000000.0,8824000000.0,8420000000.0,8809000000.0,39240000000.0,10678000000.0,11649000000.0,10752000000.0,...,17556000000.0,16425000000.0,18299000000.0,72361000000.0,22291000000.0,21870000000.0,21939000000.0,88136000000.0,24667000000.0,24108000000.0
Operating - Depreciation and Amortization,563000000.0,562000000.0,560000000.0,556000000.0,530000000.0,431000000.0,11600000000.0,434000000.0,411000000.0,356000000.0,...,633000000.0,632000000.0,612000000.0,13500000000.0,636000000.0,1300000000.0,1400000000.0,20000000000.0,1400000000.0,1500000000.0
Operating - Stock-based Compensation,973000000.0,986000000.0,969000000.0,1107000000.0,1183000000.0,1172000000.0,4652000000.0,1262000000.0,1340000000.0,1338000000.0,...,2192000000.0,2538000000.0,2465000000.0,9611000000.0,2507000000.0,2828000000.0,2703000000.0,10734000000.0,2832000000.0,3089000000.0


## I want to merge all of the accounts. This way when I am computing the different ratios all of the different accounts and cross balance sheet income statement accoutns are in the same dataframe.

In [279]:
# Pre-Processing: Clean columns names

def clean_col_names(df):
    """Removes leading/trailing whitespace from DataFrame column names."""
    df.columns = df.columns.str.strip()
    print(f"Cleaned columns: {df.columns.tolist()}") # Optional: verify cleaning
    return df

balance_sheet = clean_col_names(balance_sheet)
income_statement = clean_col_names(income_statement)
cash_flow = clean_col_names(cash_flow)

Cleaned columns: ['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik', 'Assets - Cash and Cash Equivalents', 'Assets - Short-term Investments', 'Assets - Accounts Receivable, Net (Current)', 'Assets - Accounts Receivable, Net (Non-current)', 'Assets - Inventory', 'Assets - Other Current Assets', 'Assets - Total Current Assets', 'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill', 'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets', 'Assets - Total Assets', 'Liabilities - Accounts Payable', 'Liabilities - Current Portion of Long-term Debt', 'Liabilities - Other Current Liabilities', 'Liabilities - Total Current Liabilities', 'Liabilities - Long-term Debt', 'Liabilities - Operating Lease Liabilities (Non-current)', 'Liabilities - Other Long-term Liabilities', 'Liabilities - Total Liabilities', 'Equity - Common Stock and Paid-in Capital', 'Equity - Retained Earnings', 'Equity - Accumulated Other Comprehensive Income', 'Equity - Total Stockholders

In [280]:
# Define the merge key 
# The merge key is the frame column because this column has the account data for Calendar Year or Calendar Period for different quarters
merge_key = 'frame'

# Step 1: Merge balance_sheet and income_statement
# Use 'outer' merge to keep all unique 'frame' values from both DataFrames.
# Add suffixes to distinguish overlapping columns (other than the merge key).
merged_df_1 = pd.merge(balance_sheet, income_statement,
                       on=merge_key,
                       how='outer',
                       suffixes=('_bs', '_is'))

# Doing the final merge to have the cashflow statement together with the balance sheet and income statement
merged_df_final = pd.merge(merged_df_1, cash_flow,
                           on=merge_key,
                           how='outer',
                           suffixes=('_merged', '_cf')) # Suffixes apply to cols overlapping between merged_df_1 & cash_flow






In [281]:
merged_df_final

,filed_bs,company_name_bs,end_bs,unit_bs,form_bs,frame,cik_bs,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Financing - Other Financing Activities,Financing - Net Cash from Financing Activities,Cash - Effect of Exchange Rate Changes,Cash - Net Change in Cash and Cash Equivalents,"Cash - Cash and Cash Equivalents, Beginning of Period","Cash - Cash and Cash Equivalents, End of Period",Supplemental - Interest Paid,Supplemental - Income Taxes Paid,Validation - Cash Reconciliation Difference,Validation - Net Cash Components Difference
0,2019-08-01,MICROSOFT CORPORATION,2017-09-30,USD,10-K,CY2017Q3,789019,6.884000e+09,NaN,1.456100e+10,...,-1.500000e+08,-6.341000e+09,26000000.0,NaN,6.884000e+09,6.884000e+09,NaN,NaN,NaN,NaN
1,2019-08-01,MICROSOFT CORPORATION,2017-12-31,USD,10-K,CY2017Q4,789019,1.285900e+10,NaN,1.842800e+10,...,-1.560000e+08,-1.552000e+09,-17000000.0,NaN,1.285900e+10,1.285900e+10,NaN,NaN,NaN,NaN
2,2019-08-01,MICROSOFT CORPORATION,2018-03-31,USD,10-K,CY2018Q1,789019,9.221000e+09,NaN,1.720800e+10,...,-6.400000e+08,-1.965800e+10,25000000.0,NaN,9.221000e+09,9.221000e+09,NaN,NaN,NaN,NaN
3,2020-07-31,MICROSOFT CORPORATION,2018-09-30,USD,10-K,CY2018Q3,789019,1.513700e+10,1.207430e+11,1.739000e+10,...,-7.800000e+08,-7.384000e+09,-129000000.0,3.191000e+09,1.513700e+10,1.513700e+10,NaN,NaN,-3.191000e+09,0.0
4,2020-07-31,MICROSOFT CORPORATION,2018-12-31,USD,10-K,CY2018Q4,789019,6.638000e+09,1.210240e+11,1.968000e+10,...,-4.590000e+08,-1.321600e+10,17000000.0,-8.499000e+09,6.638000e+09,6.638000e+09,NaN,NaN,8.499000e+09,0.0
5,2019-08-01,MICROSOFT CORPORATION,2019-06-30,USD,10-K,CY2019,789019,1.135600e+10,1.224630e+11,2.952400e+10,...,-6.750000e+08,-3.688700e+10,-115000000.0,-5.900000e+08,1.135600e+10,1.135600e+10,2.400000e+09,8.400000e+09,5.900000e+08,0.0
6,2020-07-31,MICROSOFT CORPORATION,2019-03-31,USD,10-K,CY2019Q1,789019,1.121200e+10,1.204060e+11,1.926900e+10,...,4.040000e+08,-7.601000e+09,18000000.0,4.574000e+09,1.121200e+10,1.121200e+10,NaN,NaN,-4.574000e+09,0.0
7,2021-07-29,MICROSOFT CORPORATION,2019-09-30,USD,10-K,CY2019Q3,789019,1.311700e+10,1.235190e+11,1.908700e+10,...,2.860000e+08,-1.020900e+10,-72000000.0,1.761000e+09,1.311700e+10,1.311700e+10,NaN,NaN,-1.761000e+09,0.0
8,2021-07-29,MICROSOFT CORPORATION,2019-12-31,USD,10-K,CY2019Q4,789019,8.864000e+09,1.253890e+11,2.352500e+10,...,-3.900000e+07,-8.915000e+09,18000000.0,-4.253000e+09,8.864000e+09,8.864000e+09,NaN,NaN,4.253000e+09,0.0
9,2020-07-31,MICROSOFT CORPORATION,2020-06-30,USD,10-K,CY2020,789019,1.357600e+10,1.229510e+11,3.201100e+10,...,-3.340000e+08,-4.603100e+10,-201000000.0,2.220000e+09,1.357600e+10,1.357600e+10,2.400000e+09,1.250000e+10,-2.220000e+09,0.0


In [282]:
import pandas as pd
import numpy as np

def calculate_cross_statement_ratios(df):
    """
    Calculate financial ratios that use both balance sheet and income statement accounts.
    For a single company with data already sorted chronologically.
    
    Parameters:
    df (DataFrame): DataFrame containing both balance sheet and income statement accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Since we have one company with data already sorted by date,
    # we can directly calculate previous period values without grouping
    result_df['Prev_Assets_Total_Assets'] = result_df['Assets - Total Assets'].shift(1)
    result_df['Prev_Equity_Total_Stockholders_Equity'] = result_df['Equity - Total Stockholders Equity'].shift(1)
    result_df['Prev_Assets_Inventory'] = result_df['Assets - Inventory'].shift(1)
    result_df['Prev_Assets_Accounts_Receivable_Current'] = result_df['Assets - Accounts Receivable, Net (Current)'].shift(1)
    result_df['Prev_Liabilities_Accounts_Payable'] = result_df['Liabilities - Accounts Payable'].shift(1)
    result_df['Prev_Assets_Total_Current_Assets'] = result_df['Assets - Total Current Assets'].shift(1)
    result_df['Prev_Liabilities_Total_Current_Liabilities'] = result_df['Liabilities - Total Current Liabilities'].shift(1)
    
    # Calculate average values
    result_df['Avg_Total_Assets'] = (result_df['Assets - Total Assets'] + result_df['Prev_Assets_Total_Assets']) / 2
    result_df['Avg_Stockholders_Equity'] = (result_df['Equity - Total Stockholders Equity'] + result_df['Prev_Equity_Total_Stockholders_Equity']) / 2
    result_df['Avg_Inventory'] = (result_df['Assets - Inventory'] + result_df['Prev_Assets_Inventory']) / 2
    result_df['Avg_Accounts_Receivable'] = (result_df['Assets - Accounts Receivable, Net (Current)'] + result_df['Prev_Assets_Accounts_Receivable_Current']) / 2
    result_df['Avg_Accounts_Payable'] = (result_df['Liabilities - Accounts Payable'] + result_df['Prev_Liabilities_Accounts_Payable']) / 2
    
    # The rest of the code remains the same
    # Calculate Working Capital for current and previous period
    result_df['Working_Capital'] = result_df['Assets - Total Current Assets'] - result_df['Liabilities - Total Current Liabilities']
    result_df['Prev_Working_Capital'] = result_df['Prev_Assets_Total_Current_Assets'] - result_df['Prev_Liabilities_Total_Current_Liabilities']
    result_df['Avg_Working_Capital'] = (result_df['Working_Capital'] + result_df['Prev_Working_Capital']) / 2
    
    # Calculate Total Debt (Current Portion + Long-term Debt)
    result_df['Total_Debt'] = result_df['Liabilities - Current Portion of Long-term Debt'] + result_df['Liabilities - Long-term Debt']
    
    # Calculate the ratios as before...
    result_df['Return_on_Assets'] = safe_divide(
        result_df['Net Income - Net Income'],
        result_df['Avg_Total_Assets']
    )
    
    result_df['Return_on_Equity'] = safe_divide(
        result_df['Net Income - Net Income'],
        result_df['Avg_Stockholders_Equity']
    )
    
    result_df['Asset_Turnover_Ratio'] = safe_divide(
        result_df['Revenue - Total Revenue'],
        result_df['Avg_Total_Assets']
    )
    
    result_df['Inventory_Turnover_Ratio'] = safe_divide(
        result_df['Cost - Total Cost of Revenue'],
        result_df['Avg_Inventory']
    )
    
    result_df['Accounts_Receivable_Turnover_Ratio'] = safe_divide(
        result_df['Revenue - Total Revenue'],
        result_df['Avg_Accounts_Receivable']
    )
    
    result_df['Accounts_Payable_Turnover_Ratio'] = safe_divide(
        result_df['Cost - Total Cost of Revenue'],
        result_df['Avg_Accounts_Payable']
    )
    
    result_df['Working_Capital_Turnover_Ratio'] = safe_divide(
        result_df['Revenue - Total Revenue'],
        result_df['Avg_Working_Capital']
    )
    
    result_df['Debt_to_EBITDA_Ratio'] = safe_divide(
        result_df['Total_Debt'],
        result_df['Operating - Operating Income']
    )
    
    return result_df


In [283]:
# Example usage:
financial_data_with_ratios = calculate_cross_statement_ratios(merged_df_final)


In [284]:
financial_data_with_ratios.to_csv('./ratios.csv')

In [285]:
merged_df_final.columns

Index(['filed_bs', 'company_name_bs', 'end_bs', 'unit_bs', 'form_bs', 'frame',
       'cik_bs', 'Assets - Cash and Cash Equivalents',
       'Assets - Short-term Investments',
       'Assets - Accounts Receivable, Net (Current)',
       'Assets - Accounts Receivable, Net (Non-current)', 'Assets - Inventory',
       'Assets - Other Current Assets', 'Assets - Total Current Assets',
       'Assets - Property, Plant and Equipment, Net', 'Assets - Goodwill',
       'Assets - Intangible Assets, Net', 'Assets - Other Long-term Assets',
       'Assets - Total Assets', 'Liabilities - Accounts Payable',
       'Liabilities - Current Portion of Long-term Debt',
       'Liabilities - Other Current Liabilities',
       'Liabilities - Total Current Liabilities',
       'Liabilities - Long-term Debt',
       'Liabilities - Operating Lease Liabilities (Non-current)',
       'Liabilities - Other Long-term Liabilities',
       'Liabilities - Total Liabilities',
       'Equity - Common Stock and Paid-in 

### Cleaning up the duplicate metadata columns created during the merge process

In [286]:
# Metadata columns (excluding the merge key 'frame') that might be duplicated
metadata_cols_to_clean = ['filed', 'company_name', 'end', 'unit', 'form', 'cik']
# Define the possible suffixes added during merges
suffixes = ['_bs', '_is', '_merged', '_cf'] # Ensure all used suffixes are listed

print("\n--- Cleaning up metadata columns ---")

for col in metadata_cols_to_clean:
    # Find all versions of the column (original name or with suffixes)
    col_versions = [c for c in merged_df_final.columns if c == col or any(c == col + s for s in suffixes)]

    if not col_versions:
        # print(f"Metadata column '{col}' not found.")
        continue

    if len(col_versions) == 1:
        # print(f"Metadata column '{col}' has no duplicates.")
        # Ensure the single version is named correctly (without suffix)
        if col_versions[0] != col and col_versions[0] in merged_df_final.columns:
             merged_df_final.rename(columns={col_versions[0]: col}, inplace=True)
        continue

    # print(f"Coalescing column: {col} from versions: {col_versions}")
    # Identify the base column name (could be the first one found, might have a suffix initially)
    base_col_name = col_versions[0]
    # Ensure the target column exists with the correct base name
    if base_col_name != col and base_col_name in merged_df_final.columns:
         merged_df_final.rename(columns={base_col_name: col}, inplace=True)
    elif base_col_name == col and col not in merged_df_final.columns:
         # This case should ideally not happen if col_versions is not empty, but handle defensively
         print(f"Warning: Base column '{col}' expected but not found directly.")
         continue # Skip if the intended target column isn't there after potential rename

    # Use fillna to fill NaNs in the target column 'col' using other versions
    for version in col_versions[1:]:
        if version in merged_df_final.columns: # Check if the source version column exists
            # print(f"  Filling NaNs in '{col}' using '{version}'")
            # Use fillna to populate missing values in 'col' from 'version'
            merged_df_final[col] = merged_df_final[col].fillna(merged_df_final[version])
        # else: # Optional: print if a version column is missing (might happen with outer merges)
            # print(f"  Version column '{version}' not found for filling '{col}'.")


    # Drop the now redundant source columns (all except the target 'col')
    cols_to_drop = [v for v in col_versions if v != col and v in merged_df_final.columns]
    if cols_to_drop:
      # print(f"  Dropping redundant columns: {cols_to_drop}")
      merged_df_final = merged_df_final.drop(columns=cols_to_drop)


--- Cleaning up metadata columns ---


# Ratio Analysis
### Some of the accounts cells are NaN. When computing the ratios that might cause a Divide by Zero error or a NaN error. For those scenarios I will define a function to safely divide by zero 

In [287]:
import pandas as pd
import numpy as np

def calculate_balance_sheet_ratios(df):
    """
    Calculate financial ratios from balance sheet accounts.
    
    Parameters:
    df (DataFrame): DataFrame containing the balance sheet accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Liquidity Ratios
    
    # Current Ratio = Current Assets / Current Liabilities
    result_df['Current_Ratio'] = safe_divide(
        result_df['Assets - Total Current Assets'],
        result_df['Liabilities - Total Current Liabilities']
    )
    
    # Quick Ratio = (Current Assets - Inventory) / Current Liabilities
    result_df['Quick_Ratio'] = safe_divide(
        (result_df['Assets - Total Current Assets'] - result_df['Assets - Inventory']),
        result_df['Liabilities - Total Current Liabilities']
    )
    
    # Cash Ratio = (Cash + Cash Equivalents) / Current Liabilities
    result_df['Cash_Ratio'] = safe_divide(
        result_df['Assets - Cash and Cash Equivalents'],
        result_df['Liabilities - Total Current Liabilities']
    )
    
    # Solvency Ratios
    
    # Debt-to-Equity Ratio = Total Liabilities / Total Shareholders' Equity
    result_df['Debt_to_Equity_Ratio'] = safe_divide(
        result_df['Liabilities - Total Liabilities'],
        result_df['Equity - Total Stockholders Equity']
    )
    
    # Debt-to-Assets Ratio = Total Liabilities / Total Assets
    result_df['Debt_to_Assets_Ratio'] = safe_divide(
        result_df['Liabilities - Total Liabilities'],
        result_df['Assets - Total Assets']
    )
    
    # Equity Ratio = Total Shareholders' Equity / Total Assets
    result_df['Equity_Ratio'] = safe_divide(
        result_df['Equity - Total Stockholders Equity'],
        result_df['Assets - Total Assets']
    )
    
    # Other Balance Sheet Metrics
    
    # Working Capital = Current Assets - Current Liabilities
    # For subtraction, NaN values will be handled automatically by Pandas
    result_df['Working_Capital'] = result_df['Assets - Total Current Assets'] - result_df['Liabilities - Total Current Liabilities']
    
    return result_df

# Example usage:
# Assuming 'financial_data' is a DataFrame containing the balance sheet accounts
financial_data_with_ratios = calculate_balance_sheet_ratios(balance_sheet)

In [288]:
financial_data_with_ratios.head(30)

,filed,company_name,end,unit,form,frame,cik,Assets - Cash and Cash Equivalents,Assets - Short-term Investments,"Assets - Accounts Receivable, Net (Current)",...,Validation - Current Assets Difference,Validation - Current Liabilities Difference,Validation - A = L+E Difference,Current_Ratio,Quick_Ratio,Cash_Ratio,Debt_to_Equity_Ratio,Debt_to_Assets_Ratio,Equity_Ratio,Working_Capital
52,2019-08-01,MICROSOFT CORPORATION,2017-09-30,USD,10-K,CY2017Q3,789019,6.884000e+09,NaN,1.456100e+10,...,1.416820e+11,4.304500e+10,0.0,3.119849,3.057638,0.133372,1.778643,0.640112,0.359888,1.094160e+11
53,2019-08-01,MICROSOFT CORPORATION,2017-12-31,USD,10-K,CY2017Q4,789019,1.285900e+10,NaN,1.842800e+10,...,1.447830e+11,4.686600e+10,0.0,2.885299,2.850824,0.221329,2.267011,0.693910,0.306090,1.095340e+11
54,2019-08-01,MICROSOFT CORPORATION,2018-03-31,USD,10-K,CY2018Q1,789019,9.221000e+09,NaN,1.720800e+10,...,1.343540e+11,3.518600e+10,0.0,3.395812,3.350638,0.199879,2.098184,0.677230,0.322770,1.105260e+11
57,2020-07-31,MICROSOFT CORPORATION,2018-09-30,USD,10-K,CY2018Q3,789019,1.513700e+10,1.207430e+11,1.739000e+10,...,1.394940e+11,4.182700e+10,0.0,2.917622,2.853404,0.268973,1.996720,0.666302,0.333698,1.079180e+11
58,2020-07-31,MICROSOFT CORPORATION,2018-12-31,USD,10-K,CY2018Q4,789019,6.638000e+09,1.210240e+11,1.968000e+10,...,1.296230e+11,3.850500e+10,0.0,3.117652,3.078680,0.131921,1.809776,0.644100,0.355900,1.065560e+11
59,2020-07-31,MICROSOFT CORPORATION,2019-03-31,USD,10-K,CY2019Q1,789019,1.121200e+10,1.204060e+11,1.926900e+10,...,1.335690e+11,3.950900e+10,0.0,2.968512,2.932289,0.208165,1.775352,0.639685,0.360315,1.060260e+11
61,2019-08-01,MICROSOFT CORPORATION,2019-06-30,USD,10-K,CY2019,789019,1.135600e+10,1.224630e+11,2.952400e+10,...,1.358820e+11,5.455300e+10,0.0,2.528839,2.499121,0.163584,1.800313,0.642897,0.357103,1.061320e+11
62,2021-07-29,MICROSOFT CORPORATION,2019-09-30,USD,10-K,CY2019Q3,789019,1.311700e+10,1.235190e+11,1.908700e+10,...,1.392580e+11,4.659400e+10,0.0,2.854468,2.809353,0.225696,1.630137,0.619792,0.380208,1.077780e+11
63,2021-07-29,MICROSOFT CORPORATION,2019-12-31,USD,10-K,CY2019Q4,789019,8.864000e+09,1.253890e+11,2.352500e+10,...,1.360760e+11,4.426200e+10,0.0,2.801375,2.770808,0.148625,1.568310,0.610639,0.389361,1.074340e+11
64,2021-07-29,MICROSOFT CORPORATION,2020-03-31,USD,10-K,CY2020Q1,789019,1.171000e+10,1.259160e+11,2.269900e+10,...,1.392700e+11,4.580800e+10,0.0,2.904338,2.876335,0.199465,1.492983,0.598874,0.401126,1.117980e+11


In [289]:
income_statement.columns

Index(['filed', 'company_name', 'end', 'unit', 'form', 'frame', 'cik',
       'Revenue - Total Revenue', 'Cost - Total Cost of Revenue',
       'Gross - Gross Profit', 'Operating Expenses - Research and Development',
       'Operating Expenses - Sales and Marketing',
       'Operating Expenses - General and Administrative',
       'Operating Expenses - Selling, General and Administrative',
       'Operating Expenses - Amortization of Intangible Assets',
       'Operating Expenses - Restructuring Charges',
       'Operating - Operating Income', 'Other - Interest Expense',
       'Other - Gain/Loss on Investments',
       'Other - Foreign Currency Transaction Gain/Loss',
       'Other - Other Income/Expense, Net', 'Tax - Income Tax Expense/Benefit',
       'Net Income - Net Income', 'EPS - Basic EPS', 'EPS - Diluted EPS',
       'Income Before Tax - Income Before Income Taxes (calculated)',
       'Validation - Gross Profit Difference'],
      dtype='object')

In [290]:
import pandas as pd
import numpy as np

def calculate_income_statement_ratios(df):
    """
    Calculate financial ratios from income statement accounts.
    
    Parameters:
    df (DataFrame): DataFrame containing the income statement accounts
    
    Returns:
    DataFrame: Original DataFrame with added ratio columns
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to safely calculate ratios, handling division by zero and NaN
    def safe_divide(numerator, denominator):
        # Perform the division
        result = numerator / denominator
        # Replace infinite values (from division by zero) with NaN
        result.replace([np.inf, -np.inf], np.nan, inplace=True)
        return result
    
    # Profitability Ratios
    
    # Gross Profit Margin = Gross Profit / Revenue
    result_df['Gross_Profit_Margin'] = safe_divide(
        result_df['Gross - Gross Profit'],
        result_df['Revenue - Total Revenue']
    )
    
    # Operating Profit Margin = Operating Income / Revenue
    result_df['Operating_Profit_Margin'] = safe_divide(
        result_df['Operating - Operating Income'],
        result_df['Revenue - Total Revenue']
    )
    
    # Net Profit Margin = Net Income / Revenue
    result_df['Net_Profit_Margin'] = safe_divide(
        result_df['Net Income - Net Income'],
        result_df['Revenue - Total Revenue']
    )
    
    # Interest Coverage Ratio = Operating Income / Interest Expense
    result_df['Interest_Coverage_Ratio'] = safe_divide(
        result_df['Operating - Operating Income'],
        result_df['Other - Interest Expense']
    )
    
    return result_df

# Example usage:
# Assuming 'income_data' is a DataFrame containing the income statement accounts
income_data_with_ratios = calculate_income_statement_ratios(income_statement)

income_data_with_ratios.head(30)

,filed,company_name,end,unit,form,frame,cik,Revenue - Total Revenue,Cost - Total Cost of Revenue,Gross - Gross Profit,...,Tax - Income Tax Expense/Benefit,Net Income - Net Income,EPS - Basic EPS,EPS - Diluted EPS,Income Before Tax - Income Before Income Taxes (calculated),Validation - Gross Profit Difference,Gross_Profit_Margin,Operating_Profit_Margin,Net_Profit_Margin,Interest_Coverage_Ratio
52,2019-08-01,MICROSOFT CORPORATION,2017-09-30,USD,10-K,CY2017Q3,789019,2.453800e+10,8.278000e+09,1.626000e+10,...,1.408000e+09,6.576000e+09,0.85,0.84,7.984000e+09,0.0,0.662646,0.314125,0.267993,11.470238
53,2019-08-01,MICROSOFT CORPORATION,2017-12-31,USD,10-K,CY2017Q4,789019,2.891800e+10,1.106400e+10,1.785400e+10,...,1.547100e+10,-6.302000e+09,-0.82,-0.82,9.169000e+09,0.0,0.617401,0.300124,-0.217927,12.434097
54,2019-08-01,MICROSOFT CORPORATION,2018-03-31,USD,10-K,CY2018Q1,789019,2.681900e+10,9.269000e+09,1.755000e+10,...,1.217000e+09,7.424000e+09,0.96,0.95,8.641000e+09,0.0,0.654387,0.309184,0.276819,12.000000
57,2020-07-31,MICROSOFT CORPORATION,2018-09-30,USD,10-K,CY2018Q3,789019,2.908400e+10,9.905000e+09,1.917900e+10,...,1.397000e+09,8.824000e+09,1.15,1.14,1.022100e+10,0.0,0.659435,0.342284,0.303397,14.770030
58,2020-07-31,MICROSOFT CORPORATION,2018-12-31,USD,10-K,CY2018Q4,789019,3.247100e+10,1.242300e+10,2.004800e+10,...,1.965000e+09,8.420000e+09,1.09,1.08,1.038500e+10,0.0,0.617412,0.315913,0.259308,15.264881
59,2020-07-31,MICROSOFT CORPORATION,2019-03-31,USD,10-K,CY2019Q1,789019,3.057100e+10,1.017000e+10,2.040100e+10,...,1.677000e+09,8.809000e+09,1.15,1.14,1.048600e+10,0.0,0.667332,0.338262,0.288149,15.411326
61,2019-08-01,MICROSOFT CORPORATION,2019-06-30,USD,10-K,CY2019,789019,1.258430e+11,4.291000e+10,8.293300e+10,...,4.448000e+09,3.924000e+10,5.11,5.06,4.368800e+10,0.0,0.659020,0.341370,0.311817,15.993671
62,2021-07-29,MICROSOFT CORPORATION,2019-09-30,USD,10-K,CY2019Q3,789019,3.305500e+10,1.040600e+10,2.264900e+10,...,2.008000e+09,1.067800e+10,1.40,1.38,1.268600e+10,0.0,0.685191,0.383785,0.323037,19.915228
63,2021-07-29,MICROSOFT CORPORATION,2019-12-31,USD,10-K,CY2019Q4,789019,3.690600e+10,1.235800e+10,2.454800e+10,...,2.436000e+09,1.164900e+10,1.53,1.51,1.408500e+10,0.0,0.665149,0.376389,0.315640,21.240061
64,2021-07-29,MICROSOFT CORPORATION,2020-03-31,USD,10-K,CY2020Q1,789019,3.502100e+10,1.097500e+10,2.404600e+10,...,2.091000e+09,1.075200e+10,1.41,1.40,1.284300e+10,0.0,0.686617,0.370492,0.307016,21.131922


#### There are some ratios that are not well computed because they need computation accross different time period but it will not work for the first period so you need a way out of it. You could shift the whole dataframe or you could start the computation on the second one. 

- Note: Ratios requiring averages (like ROA, ROE, Turnovers, DSO, DIO, DPO) are calculated
- using end-of-period values here for simplicity. For more accurate analysis,
- calculate average balances: (balance_start_period + balance_end_period) / 2.
- This requires having data from the previous period (e.g., using df.shift()).

### In what follows I have defined different functions that compute Liquidity Ratios